In [7]:
! pip install -q -U langchain-ollama  python-dotenv gradio langchain plotly scikit-learn matplotlib langchain-community
! pip install pymilvus -q

In [8]:
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [9]:
# imports for langchain, plotly and Chroma
from langchain.vectorstores import Milvus
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_ollama import OllamaEmbeddings, ChatOllama

In [ ]:
# # Load environment variables in a file called .env
# load_dotenv(override=True)
# os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
# ! ollama pull nomic-embed-text
# ! ollama pull mistral-openorca
! ollama pull phi4-mini
! ollama serve

In [ ]:
! podman ps

In [50]:
! cd .. && podman compose up -d

>>>> Executing external compose provider "/opt/homebrew/bin/docker-compose". Please see podman-compose(1) for how to disable this message. <<<<

WARN[0000] /Users/adsonalves/Desktop/FIT/Prediza/docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion 
[+] Running 3/0
 ✔ Network milvus               Created                                    0.0s 
 ✔ Container milvus-etcd        Created                                    0.1s 
 ✔ Container milvus-minio       Created                                    0.1s 
 ⠋ Container milvus-standalone  Creating                                   0.0s 
[+] Running 2/4
 ✔ Network milvus               Created                                    0.0s 
 ⠙ Container milvus-etcd        Starting                                   0.2s 
 ⠙ Container milvus-minio       Starting                                   0.2s 
 ✔ Container milvus-standalone  Created                                    0.0s 
[+

In [10]:
! podman ps

CONTAINER ID  IMAGE                             COMMAND               CREATED        STATUS        PORTS                                             NAMES
1502ec8eda38  docker.io/library/postgres:15     postgres              5 minutes ago  Up 5 minutes  0.0.0.0:5433->5432/tcp                            postgres
94b3d6e966bd  quay.io/minio/minio:latest        server /data --co...  5 minutes ago  Up 5 minutes  0.0.0.0:9000-9001->9000-9001/tcp                  minio
6a5410eda16d  quay.io/coreos/etcd:v3.5.5        /usr/local/bin/et...  5 minutes ago  Up 5 minutes  0.0.0.0:2379-2380->2379-2380/tcp                  etcd
7f0d49f4b498  docker.io/bitnami/spark:3.5       /opt/bitnami/scri...  5 minutes ago  Up 5 minutes  0.0.0.0:7077->7077/tcp, 0.0.0.0:8080->8080/tcp    spark
6e783e464708  docker.io/ollama/ollama:latest    serve                 5 minutes ago  Up 5 minutes  0.0.0.0:11434->11434/tcp                          ollama
a171887c1445  docker.io/milvusdb/milvus:v2.3.9  milvus run standa..

In [11]:
from pymilvus import connections, utility

# Conecte-se ao Milvus na porta 19530
connections.connect(alias="default", host="localhost", port="19530")

# Agora você pode listar coleções
collections = utility.list_collections()
print("Coleções:", collections)

Coleções: []


In [13]:
from pymilvus import connections

status = connections.has_connection("default")
print("Conectado:", status)


Conectado: True


In [ ]:
# !  cd .. && podman compose down && rm -rf volumes

In [58]:
# MODEL = "gpt-4o-mini"
MODEL = "nomic-embed-text"

In [59]:
folders = glob.glob("../knowledge_base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []

for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")

Total number of chunks: 17
Document types found: {'services', 'company', 'infos'}


In [60]:
embeddings = OllamaEmbeddings(model=MODEL)

In [61]:
vector_store = Milvus.from_documents(
    chunks,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": "19530"},
    collection_name="knowledge_base_prediza"
)

In [70]:
from pymilvus import connections, utility

# connections.connect(alias="default", host="localhost", port="19530")

collections = utility.list_collections()
print("Coleções no Milvus:")
for collection in collections:
    print("-", collection)

Coleções no Milvus:
- knowledge_base_prediza


In [67]:
## Resolveu o problema de carregamento no DB Milvus --- erro (aparece nulo mas ta la, confia)
from pymilvus import Collection

collection = Collection("knowledge_base_prediza")
collection.load()
print("Coleção carregada com sucesso.")
print("Número de vetores na coleção:", collection.num_entities)

Coleção carregada com sucesso.
Número de vetores na coleção: 0


In [69]:
from pymilvus import Collection

collection_name = "knowledge_base_prediza"
collection = Collection(name=collection_name)

num_documents = collection.num_entities
print(f"A coleção '{collection_name}' tem {num_documents} documentos.")


A coleção 'knowledge_base_prediza' tem 0 documentos.


In [ ]:
# # para deletar todas colecoes no milvus 

# from pymilvus import connections, utility

# # Conecte-se ao Milvus
# # connections.connect(alias="default", host="localhost", port="19530")

# # Liste todas as coleções
# collections = utility.list_collections()

# # Apague cada coleção
# for collection_name in collections:
#     print(f"Apagando a coleção: {collection_name}")
#     utility.drop_collection(collection_name)

# # Verifique novamente as coleções após a remoção
# remaining_collections = utility.list_collections()
# print("Coleções restantes:", remaining_collections)

Apagando a coleção: knowledge_base_prediza
Coleções restantes: []


In [71]:
from sklearn.decomposition import PCA
import plotly.express as px
import pandas as pd

# Gera embeddings e metadados
emb_vectors = [embeddings.embed_query(chunk.page_content) for chunk in chunks]
texts = [chunk.page_content for chunk in chunks]
doc_types = [chunk.metadata.get("doc_type", "desconhecido") for chunk in chunks]

# Reduz os vetores para 2D
pca = PCA(n_components=2)
emb_2d = pca.fit_transform(emb_vectors)

# Cria DataFrame
df = pd.DataFrame({
    "x": emb_2d[:, 0],
    "y": emb_2d[:, 1],
    "text": texts,
    "doc_type": doc_types
})

# Plota com Plotly, colorido por tipo de documento
fig = px.scatter(
    df, x="x", y="y",
    color="doc_type",
    hover_data=["text"],
    title="Visualização dos Embeddings por Tipo de Documento"
)
fig.show()

In [72]:

# Reduz para 3D
pca = PCA(n_components=3)
emb_3d = pca.fit_transform(emb_vectors)

# Cria DataFrame
df = pd.DataFrame({
    "x": emb_3d[:, 0],
    "y": emb_3d[:, 1],
    "z": emb_3d[:, 2],
    "text": texts,
    "doc_type": doc_types
})

# Gráfico 3D com Plotly
fig = px.scatter_3d(
    df, x="x", y="y", z="z",
    color="doc_type",
    hover_data=["text"],
    title="Visualização 3D dos Embeddings por Tipo de Documento"
)

fig.show()

In [73]:
# create a new Chat with OpenAI
llm = ChatOllama(temperature=0.7, model="phi4")
# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vector_store.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)


In [74]:
# Let's try a simple question
query = "O que ou quem é Prediza ?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

A Prediza é uma empresa que oferece serviços baseados em análise preditiva e visão computacional para diferentes setores. Nos serviços de Agricultura Preditiva, a Prediza utiliza tecnologia avançada para fornecer informações em tempo real sobre riscos, melhorando a segurança no manejo agrícola, reduzindo custos e aumentando a produtividade. Isso contribui tanto para o desempenho do negócio quanto para a proteção ambiental.

Além disso, a Prediza oferece serviços de monitoramento de ruído ambiental, que são essenciais para cidades, indústrias e empresas que trabalham com geração de energia. Com suas soluções, a empresa ajuda na coleta e análise de dados acústicos, auxiliando nas decisões estratégicas para suprimir riscos relacionados à poluição sonora e aumentar a eficiência produtiva. Isso inclui monitoramento contínuo de ruído, simulação acústica e geração de mapas de ruído baseados em curvas isofônicas no ambiente.


In [75]:
# Wrapping that in a function
system_message = "Você é especialista em responder perguntas precisas sobre a Prediza, uma empresa de ciência da computação aplicada na agricultura, energia e meio ambiente. Seja breve e preciso. Se não souber a resposta, diga. Não invente nada se não tiver recebido contexto relevante."

def chat(question,system_message=system_message):
    messages = [{"role": "system", "content": system_message}]
    messages.append({"role": "user", "content": question})
    result = conversation_chain.invoke({"question": messages})
    return result["answer"]

In [76]:
# And in Gradio:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
